## Introduction
The software aims to provide users with functions that solve derivatives using automatic differentiation. Solving derivatives is an important task in many fields, as derivatives allow for the optimization of functions and for precise descriptions of rates of change. For example, engineers use derivatives to optimize space given material constraints, physicists use derivatives to map particle motion, and economists use derivatives to describe changing markets.

We will implement automatic differentiation (AD). AD is a powerful method, as it allows users to solve derivatives for highly complex functions efficiently and with machine precision.

## Background

The chain rule is used to evaluate the derivative of a function by differentiating the outer function, multiplied by the derivative of the inner function, until all layers of the function are complete.

Suppose we have a function $F(x) = f(g(x))$, then $F'(x) = f'(g(x))g'(x)$.

Using Leibniz notation, the chain rule is represented as:

$$\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}$$


We can traverse a function $f(x)$ by following what happens to its input, $x$. We evaluate from the innermost layer and work our way outwards. For each layer, we calculate its derivative and then feed that as the input to the next layer.

Elementary functions (_eg._ $sin$, $cos$, $exp$, $log$) are common in AD applications. These functions usually differentiate to fixed solutions; therefore, we implement methods that evaluate these functions.

## How to use socialAD

Users can simply download our open-source package _socialAD_ via pip. 


```
pip install socialAD
```

Then, to get the forward mode implementation in you work environment, use `from socialAD.forward import *`. Importing numpy is also required.

To instantiate AD objects, we can simply call `forwardAD(val)` with an initial function input value.

Here is a quick demonstration of using the forward mode to solve the derivative of the function $$f(x) = e^{x} - (2 - 6x - 3x^{5})$$ at x = 5. Note: all multiplication must be made explicit.
For more information on writing functions for our package, see the "Elementary function conventions" section below.


In [ ]:
from socialAD.forward import *
import numpy as np

#Instantiate initial input to function
x = forwardAD(5)

#Function (note: make multiplication explicit)
f = e()**x - (2 - 6*x - 3 * x ** 5)

#Get function value at x = 5 and derivative at x = 5
print('Function value: ', f.val, '\nDerivative value: ', f.der)

Function value:  9551.413159102576 
Derivative value:  9529.413159102576


Here is a demonstration of finding a root for that function using Newton's Method - our package is used for finding the derivatives within Newton's Method:

In [ ]:
#Initialize root value and step value
root = 0.0
dx = 50

#While step is greater than a very small number
while dx > 1e-20:

    #Instantiate a forward AD object at root guess
	x = forwardAD(root)

    #Write our function
	f = e()**x - (2 - 6*x - 3 * x ** 5)

    #Get step using Newton's Method
	dx = -f.val / f.der

    #Save new root with new step
	root = root + dx
    
print('Root value: ', root, '\nFunction value at root: ', f.val)

Root value:  0.14133666319971333 
Function value at root:  0.010886349092763536


## Software Organization


**Directory Structure:**

`docs`

- Includes documentation, including these subdirectories:

`examples` ** has not been implemented yet

- Includes example use in social science, including these subdirectories:

- `simple_chain_rule`

- `economic_rate_of_change`

- `social_growth_rate`

`socialAD`

- Includes all install and method materials
- So far, we have `forward.py`

`tests`
- Include all the tests (currently, the most relevant is: `forwardAD_tests.py`)

README.md

- Includes install instructions, author names, and CodeCov/TravisCI buttons

requirements.txt

- Lists requirements for being able to successfully use our package


**Here are the modules and their basic functionality**
- forward: computes derivative using forward mode of automatic differentiation
- social: computes maximum or minimum of a loss or objective function for applications in social science

**Our suite live is tested on both TravisCI and CodeCov**
Please check `.travis.yml` to see how the tests were performed. The big picture us the suite is built on Travis, and CodeCov runs the testing scripts in the `tests` folder. CodeCov counts the numbers of lines that the program hits.
The percentage and the fact whether the package is  built successfully will reflect on the CodeCov and Travis badges.

**Our package will be available for install from PyPI, and can be installed, updated, or removed using pip3**

**Framework for software packaging**
- We will package our software using the PyScaffold framework, as it will help facilitate configuration, versioning, and has some helpful extensions 
such as compatibility with Django and compatibility with Sphinx. For documentation, we will use the Sphinx framework.

**Notes**
- Our software will be designed in Python3 and will run in Python3 environments
- Our software is pip installable: `pip install socialAD`

## Implementation

Implementation of the _forward mode_ of automatic differentiation.

* Core data structures:

Data will be stored in `numpy.array`. Numpy is the only external dependency of the package.

* Classes:

    * `forwardAD`: the main automatic differentiation object. The classes below are subclasses of `forwardAD`.
        * Instantiate with a value and input into a function to find function value and derivative value.
        * Attributes:
            * `.der`: derivative as evaluated at the input value
            * `.val`: function value as evaluated at the inpute value
        * Example (to represent the function $3 + 2x$):
            ```
            >>> x = fowardAD(5)
            >>> f = 3 + 2 * x
            >>> print(f.val, f.der)
            13.0 2.0
            ```

    * `e`: represents the natural number e. Inherits from `forwardAD`
        * Instantiate with an open parenthesis to represent the natural number in a function: `e()`
        * Example (to represent the function $3 + e^{x}$):
            ```
            >>> x = fowardAD(5)
            >>> f = 3 + e() ** x
            >>> print (f.val, f.der)
            151.413159 148.413159
            ```

    * `log`: represents a logarithm. Defaults to natural logarithm (ln), but the base can be changed.
        * Instantiate with functional input inside of `log()`
        * Example (to represent the function $ln(3x) + 5$):
            ```
            >>> x = fowardAD(5)
            >>> f = log(3 * x) + 5
            >>> print (f.val, f.der)
            7.708050201 0.2
            ```
        * Example (to represent the function $\log_{10}(3x) + 5$):
            ```
            >>> x = fowardAD(5)
            >>> f = log(3 * x, base = 10) + 5
            >>> print (f.val, f.der)
            6.17609125905 0.08685889638065
            ```

    * Basic trigonometric functions: `sin` (sine), `cos` (cosine), `tan` (tangent), `arcsin` (arcsine), `arccos` (arccosine), `arctan`(arctangent), `sinh` (hyperbolic sine), `cosh` (hyperbolic cosine), `tanh` (hyperbolic tangent)
        * Trigonometric functions, all calculations done in radians
        * Instantiate with functional input inside of trigonometric class
        * Example (to represent the function $\frac{\sin(x+3)}{\arcsin(-1)}$):
            ```
            >>> x = fowardAD(5)
            >>> f = sin(x + 3) / arcsin(-1)
            >>> print (f.val, f.der)
            -0.6298 0.09263
            ```
            
    * `optimize` (hasn't been implemented yet) 
        * Will find minima and maxima of various functions used in social science algorithms (can be applied in other fields as well)

* Important methods and attributes:
    * `forwardAD` and its subclasses (`e`,`log`,`sin`, etc.) have two key attributes
        * `.val`: the function value evaluated at the provided input
        * `.der`: the derivative value evaluated at the provided input
    * In the future, the following methods will be implemented for our social science extension:
        * `differential_equation`
            * `seed` attribute
            * `evaluate` method
        * `equation`
            * `f` attribute (converting the inputs to `numpy` formats)
            * `jacobian` method
        * `optimize` 
            * `domain` attribute
            * `application` attribute
            * `max` method: calculating maximum objective/loss function
            * `min` method: calculating minimum objective/loss function


* External dependencies:

    * `numpy`

* Elementary function conventions:

    * The basic operations in python apply to our package. For example, addition can by symbolized with `+`, subtraction with `-`, multiplication with `*`, and division with `/`
    * All multiplication must be made explicity with a `*` symbol.
    * Raising to a power is symbolized with `**`
    * The natural number, e, can be used by calling the `e()` class. To exponentiate to a power (ex: to the power of x), write: `e()**x`
    * To write a logarithm, call the `log()` class. This class defaults to the natural logarith (base e), but the base can be changed by setting the `base` attribute to the desired base.
    * To write trig functions, we have the following classes:
        * `sin()` for sine
        * `cos()` for cosine
        * `tan()` for tangent
        * `arcsin()` for arcsine
        * `arccos()` for arccosine
        * `arctan()` for arctangent
        * `sinh()` for hyperbolic sine
        * `cosh()` for hyperbolic cosine
        * `tanh()` for hyperbolic tangent
    
    
    * Two helpful examples:

        * To write $f(x) = e^{x} - (2 - 6x - 3x^{5})$ we'd write: `f = e()**x - (2 - 6*x - 3 * x ** 5)`
        * to write $f(x) = ln(x) + sin(x + 5) - \frac{x^2}{\log_{10}{(12x)}}$ we'd write: `f = log(x) + sin(x + 5) - (x**2) / (log(12 * x, base = 10))`



## Future Features

To build upon the work we've already done, we will implement classes that compute the inverse of the hyperbolic trig functions. Then, we will expand the classes we've already written to handle multiple inputs for multiple variables. 

Like the rest of our implementation in `forward` we will attempt to do this through duck-typing. In addition, we will modify our code such that (when requested) it outputs a Jacobian matrix as a numpy array, for cases with multiple input variables.

For our extension, we will implement another module called `social` that will compute maxima or minima of a loss or objective function for applications in social science.

The intent behind this extension is to provide ready resources for social science programmers, who often optimize loss functions for fitting growth models or inferential models.

Unfortunately, algorithms for application in social science are often slow or difficult to use, due to the lack of programmers who focus in the social sciences (compared to other fields). Our package is unique in that it will provide a fast implementation that is marketed towards social scientists! 

Here is a list of planned methods:

* In the future, the following methods will be implemented for our social science extension:
    * `differential_equation`
        * `seed` attribute
        * `evaluate` method
    * `equation`
        * `f` attribute (converting the inputs to `numpy` formats)
        * `jacobian` method
    * `optimize` 
        * `domain` attribute
        * `application` attribute
        * `max` method: calculating maximum objective/loss function
        * `min` method: calculating minimum objective/loss function

We will also add to our documentation some examples of package use that directly relate to common uses in social science.

Some anticipated challenges are figuring out ways to handle multiple dimensions which don't reduce speed. In addition, another challenge we will face is making sure our optimization implementation gets as close to machine precision as possible.

